In [0]:
pip install azure-storage-blob

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
pip install nltk

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
pip install wordcloud

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from nltk.corpus import stopwords
import seaborn as sns


In [0]:
import requests
import pandas as pd
import json
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from azure.core.pipeline.transport import HttpResponse
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, BooleanType, ArrayType
import gzip
import plotly
import plotly.graph_objects as go
from pyspark.sql.functions import dayofmonth, countDistinct, to_date, hour
from pyspark.sql.functions import explode
from pyspark.sql.functions import lower
from pyspark.sql.functions import trim
from pyspark.sql.functions import regexp_replace, lower
from pyspark.sql.functions import length
from pyspark.sql.functions import trim
from pyspark.sql.functions import posexplode
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.ml.feature import Tokenizer
from nltk.tokenize import RegexpTokenizer
import nltk
from nltk.probability import FreqDist
from pyspark.sql.functions import col, lower, when


In [0]:
spark = SparkSession.builder.getOrCreate()

storage_account_name = "pod4projectstorage"
storage_account_access_key = "2hClDrVPLGX4QBDBk8OylAkHqczIQfDja66Yl488rmj/0+vb+CAzOxL5qMe5XyM9ZupgwveVRm3N+AStriO5vg=="

container_name = "data1"
directory_path = ""

spark.conf.set("fs.azure.account.key." + storage_account_name + ".blob.core.windows.net", storage_account_access_key)

df = spark.read.parquet("wasbs://" + container_name + "@" + storage_account_name + ".blob.core.windows.net/" + directory_path)


In [0]:
df.show()

+-----------+------+--------------------+--------------+--------------------+--------------------+-------------------------------------+------------------------+-----------------------+-----------------------------+--------------------+-------------------+--------------------+----------------------+------------------------+------------------------+-----------------------+--------------------------------+--------------------------------+---------------------------------+--------------------------+
|       type|public|          created_at|payload.action|   payload.issue.url|payload.issue.number|                  payload.issue.title|payload.issue.user.login|payload.issue.user.type|payload.issue.user.site_admin|payload.issue.labels|payload.issue.state|payload.issue.locked|payload.issue.comments|payload.issue.created_at|payload.issue.updated_at|payload.issue.closed_at|payload.issue.author_association|payload.issue.active_lock_reason|               payload.issue.body|payload.issue.state_reas

In [0]:
issues_df = df.filter(df.type=='IssuesEvent')

In [0]:
issues_df.show()

+-----------+------+--------------------+--------------+--------------------+--------------------+-------------------------------------+------------------------+-----------------------+-----------------------------+--------------------+-------------------+--------------------+----------------------+------------------------+------------------------+-----------------------+--------------------------------+--------------------------------+---------------------------------+--------------------------+
|       type|public|          created_at|payload.action|   payload.issue.url|payload.issue.number|                  payload.issue.title|payload.issue.user.login|payload.issue.user.type|payload.issue.user.site_admin|payload.issue.labels|payload.issue.state|payload.issue.locked|payload.issue.comments|payload.issue.created_at|payload.issue.updated_at|payload.issue.closed_at|payload.issue.author_association|payload.issue.active_lock_reason|               payload.issue.body|payload.issue.state_reas

In [0]:
for col_name in issues_df.columns:
    new_col_name = col_name.replace("payload.", "")
    issues_df = issues_df.withColumnRenamed(col_name, new_col_name)


In [0]:
issues_df.show()

+---------+------+--------------------+--------------------+--------------------+--------------------+------+--------------------+--------------------+-------------+------+-----+-----------+-----------+--------------------+----+--------+
|     type|public|          created_at|                 org|               actor|                repo|action|              before|             commits|repository_id|review|pages|    push_id|pusher_type|                 ref|size|ref_type|
+---------+------+--------------------+--------------------+--------------------+--------------------+------+--------------------+--------------------+-------------+------+-----+-----------+-----------+--------------------+----+--------+
|PushEvent|  true|2023-06-01T00:00:00Z|{https://avatars....|{https://avatars....|{474293004, kuda-...|  null|6f6f6712bea287e78...|[{{robbycaesar@gm...|    474293004|  null| null|13828565359|       null|  refs/heads/develop|   2|    null|
|PushEvent|  true|2023-06-01T00:00:00Z|         

In [0]:
grouped_df = df.groupBy("type").count().orderBy("count", ascending=False)

In [0]:
pd_data = grouped_df.toPandas()
pd_data

,type,count
0,PushEvent,60183436


In [0]:
fig = go.Figure([go.Bar(x=pd_data['type'], y=pd_data['count'],marker=dict(color='blue'))])
fig.update_traces( marker_line_color='rgb(8,48,107)',
                  marker_line_width=1.5, opacity=0.6)
fig.update_layout(title_text='Event Counts',xaxis=dict(tickangle=90))
fig.show()

In [0]:
push_df = issues_df

In [0]:
for col_name in df.columns:
    new_col_name = col_name.replace("payload.", "")
    push_df = push_df.withColumnRenamed(col_name, new_col_name)
push_df.show()

+---------+------+--------------------+--------------------+--------------------+--------------------+------+--------------------+--------------------+-------------+------+-----+-----------+-----------+--------------------+----+--------+
|     type|public|          created_at|                 org|               actor|                repo|action|              before|             commits|repository_id|review|pages|    push_id|pusher_type|                 ref|size|ref_type|
+---------+------+--------------------+--------------------+--------------------+--------------------+------+--------------------+--------------------+-------------+------+-----+-----------+-----------+--------------------+----+--------+
|PushEvent|  true|2023-06-01T00:00:00Z|{https://avatars....|{https://avatars....|{474293004, kuda-...|  null|6f6f6712bea287e78...|[{{robbycaesar@gm...|    474293004|  null| null|13828565359|       null|  refs/heads/develop|   2|    null|
|PushEvent|  true|2023-06-01T00:00:00Z|         

In [0]:
hour_df = push_df.withColumn('hour', hour(push_df['created_at']))

In [0]:
hour_df.show()

+---------+------+--------------------+--------------------+--------------------+--------------------+------+--------------------+--------------------+-------------+------+-----+-----------+-----------+--------------------+----+--------+----+
|     type|public|          created_at|                 org|               actor|                repo|action|              before|             commits|repository_id|review|pages|    push_id|pusher_type|                 ref|size|ref_type|hour|
+---------+------+--------------------+--------------------+--------------------+--------------------+------+--------------------+--------------------+-------------+------+-----+-----------+-----------+--------------------+----+--------+----+
|PushEvent|  true|2023-06-01T00:00:00Z|{https://avatars....|{https://avatars....|{474293004, kuda-...|  null|6f6f6712bea287e78...|[{{robbycaesar@gm...|    474293004|  null| null|13828565359|       null|  refs/heads/develop|   2|    null|   0|
|PushEvent|  true|2023-06-01

In [0]:
grouped_hour = hour_df.groupBy("hour").count().orderBy("count", ascending=False)

In [0]:
pd_hour = grouped_hour.toPandas()

In [0]:
fig = go.Figure([go.Bar(x=pd_hour['hour'], y=pd_hour['count'],marker=dict(color='orangered'))])
fig.update_traces( marker_line_color='rgb(8,48,107)',
                  marker_line_width=1.5, opacity=0.6,width=0.7)
fig.update_layout(title_text='Commits by Hour',width = 1000, height = 550)
fig.show()

In [0]:
result = push_df.groupBy(dayofmonth("created_at").alias("Day")).agg(countDistinct("actor.id").alias("Users")).orderBy("Users", ascending=False)

In [0]:
result.show()

In [0]:
pd_date = result.toPandas()

In [0]:
fig = go.Figure([go.Bar(x=pd_date['Day'], y=pd_date['Users'], marker=dict(
        color=['yellowgreen' if val == pd_date['Users'].max() else 'royalblue' for val in pd_date['Users']]))])
fig.update_traces( marker_line_color='rgb(8,48,107)',
                  marker_line_width=1.5, opacity=0.6,width=0.7)
fig.update_layout(title_text='Commits by Day of the Month',width = 1000, height = 550)
fig.show()

In [0]:
grouped_pushdf = push_df.select('org.login').dropna()

In [0]:
grouped_pushdf = grouped_pushdf.groupBy("login").count().orderBy("count", ascending=False)

In [0]:
grouped_pushdf.show()

In [0]:
pd_pushdf = grouped_pushdf.toPandas()
pd_pushdf = pd_pushdf.head()

In [0]:
pd_pushdf

In [0]:
fig = go.Figure([go.Bar(x=pd_pushdf['login'], y=pd_pushdf['count'],marker=dict(color='yellowgreen'))])
fig.update_traces( marker_line_color='rgb(8,48,107)',
                  marker_line_width=1.5, opacity=0.6,width=0.5)
fig.update_layout(title_text='Organizations with highest number of commits',width = 1000, height = 550)
fig.show()

In [0]:
exploded_msg = push_df.select(explode('commits.message').alias("message"))

In [0]:
df.show()

+---------+------+--------------------+--------------------+--------------------+--------------------+--------------+--------------------+--------------------+---------------------+--------------+-------------+---------------+-------------------+--------------------+------------+----------------+
|     type|public|          created_at|                 org|               actor|                repo|payload.action|      payload.before|     payload.commits|payload.repository_id|payload.review|payload.pages|payload.push_id|payload.pusher_type|         payload.ref|payload.size|payload.ref_type|
+---------+------+--------------------+--------------------+--------------------+--------------------+--------------+--------------------+--------------------+---------------------+--------------+-------------+---------------+-------------------+--------------------+------------+----------------+
|PushEvent|  true|2023-06-01T00:00:00Z|{https://avatars....|{https://avatars....|{474293004, kuda-...|    

In [0]:
data = issues_df

In [0]:
for col_name in data.columns:
    new_col_name = col_name.replace("payload.", "").replace('issue.','').replace('user.','')
    data = data.withColumnRenamed(col_name, new_col_name)

In [0]:
data.show()

+-----------+------+--------------------+------+--------------------+-------+-------------------------------------+--------------------+----+----------+--------------------+------+------+--------+--------------------+--------------------+--------------------+------------------+------------------+---------------------------------+------------+
|       type|public|          created_at|action|                 url| number|                                title|               login|type|site_admin|              labels| state|locked|comments|          created_at|          updated_at|           closed_at|author_association|active_lock_reason|                             body|state_reason|
+-----------+------+--------------------+------+--------------------+-------+-------------------------------------+--------------------+----+----------+--------------------+------+------+--------+--------------------+--------------------+--------------------+------------------+------------------+-------------

In [0]:
# inline -> exploding, array_ip -> to zip, selectExpr -> to run transf on multiple columns.
# Inline vs explode? 
# Can explode be used w multiple columns?
# Is this the right way of using Inline?
new_df = data.selectExpr('inline(arrays_zip(labels.name,labels.description))')

In [0]:
new_df.show()

+--------------------+--------------------+
|                name|         description|
+--------------------+--------------------+
|       documentation|Improvements or a...|
|                auth|Authentication an...|
|                 bug|                null|
|              status|                null|
|invidious-invidio...|                null|
|                task|                    |
|        nice_to_have|                    |
|              status|                null|
|             clam-av|                null|
|              status|                null|
|   h2-invent-support|                null|
|              status|                null|
| evon-stone-ceramics|                null|
|      design pattern|                    |
|              status|                null|
|    relay-froth-zone|                null|
|              status|                null|
|piped-piped-esmai...|                null|
|              status|                null|
|          paid-panel|          

In [0]:
# Can both be done in a single-stat? Yes
new_df = new_df.withColumn("description", trim(lower(regexp_replace(new_df.description, "[^a-zA-Z\\s]", "")))).withColumn("name", trim(lower(regexp_replace(new_df.name, "[^a-zA-Z\\s]", ""))))

In [0]:
new_df.show()

+--------------------+--------------------+
|                name|         description|
+--------------------+--------------------+
|       documentation|improvements or a...|
|                auth|authentication an...|
|                 bug|                null|
|              status|                null|
|invidiousinvidiou...|                null|
|                task|                    |
|          nicetohave|                    |
|              status|                null|
|              clamav|                null|
|              status|                null|
|      hinventsupport|                null|
|              status|                null|
|   evonstoneceramics|                null|
|      design pattern|                    |
|              status|                null|
|      relayfrothzone|                null|
|              status|                null|
|pipedpipedesmaile...|                null|
|              status|                null|
|           paidpanel|          

In [0]:
new_df = new_df.filter((new_df.name != '') & (new_df.description != ''))
new_df.show()

+-----------------+--------------------+
|             name|         description|
+-----------------+--------------------+
|    documentation|improvements or a...|
|             auth|authentication an...|
|              bug|bug that is not s...|
|           triage|     untriaged issue|
|        duplicate|label duplicate c...|
|   security issue|related to code s...|
|              bug|something isnt wo...|
|      enhancement|new feature or re...|
|              bug|something isnt wo...|
|      enhancement|new feature or re...|
|    triagesupport|indicates an issu...|
|  lifecyclerotten|denotes an issue ...|
|              bug|something isnt wo...|
|      enhancement|enhancement or im...|
|in vivo selection|in vivo selection...|
|    documentation|improvements or a...|
|  lifecyclerotten|denotes an issue ...|
|              bug|something is not ...|
|        confirmed|the developers ha...|
|        duplicate|label duplicate c...|
+-----------------+--------------------+
only showing top

In [0]:
name_group = new_df.groupBy(new_df.name).count().orderBy("count", ascending=False).limit(10).toPandas()

In [0]:
name_group

,name,count
0,bug,9405
1,enhancement,8619
2,documentation,1295
3,good first issue,1022
4,question,795
5,help wanted,767
6,feature,645
7,mend dependency security vulnerability,605
8,type bug,567
9,kindbug,407


In [0]:
fig = go.Figure([go.Bar(x=name_group['name'], y=name_group['count'],marker=dict(color='darkmagenta'))])
fig.update_traces( marker_line_color='rgb(8,48,107)',
                  marker_line_width=1.5, opacity=0.6,width=0.5)
fig.update_layout(title_text='Label Counts',xaxis=dict(tickangle=90))
fig.show()

In [0]:
uniq_names = new_df.select(new_df.name).distinct().toPandas()

In [0]:
uniq_names['name']

Out[208]: 0                    techissues
1                 priority high
2                      priority
3                   areageneric
4          team app development
                 ...           
5908           reported by user
5909               type tracker
5910                data update
5911         identifier handles
5912    maturity ready for work
Name: name, Length: 5913, dtype: object

In [0]:
pandas_df = new_df.select('description').toPandas()

In [0]:
# converting to lowercase
pandas_df['description'] = pandas_df['description'].astype(str).str.lower()

# converting to tokens
regexp = RegexpTokenizer('\w+')
pandas_df['description_token']=pandas_df['description'].apply(regexp.tokenize)
nltk.download('stopwords')

# stopwords 
stopwords1 = nltk.corpus.stopwords.words("english")
my_stopwords = ['something','working','team','work', 'new','issues','isnt']
stopwords1.extend(my_stopwords)
pandas_df['description_token'] = pandas_df['description_token'].apply(lambda x: [item for item in x if item not in stopwords1])

# infrequent words
#pandas_df['description_str'] = pandas_df['description_token'].apply(lambda x: ' '.join([item for item in x if len(item)>2]))

# creating list of words
all_words_description = ' '.join([word for word in pandas_df['description_token']])

# tokenizing all words
nltk.download('punkt')
tokenized_words_description = nltk.tokenize.word_tokenize(all_words_description)

# frequency distribution
fdist_description = FreqDist(tokenized_words_description)
pandas_df['description_fdist'] = pandas_df['description_token'].apply(lambda x: ' '.join([item for item in x if fdist_description[item] >= 1]))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
all_words_desc = ' '.join([word for word in pandas_df['description_fdist']])

In [0]:
# fd table
words = nltk.word_tokenize(all_words_desc)
fd = FreqDist(words)
data_fd = list(fd.items())
df = pd.DataFrame(data_fd, columns=['Word', 'Count'])
df = df.sort_values(by='Count', ascending=False).head(6)
df = df.sort_values(by='Count', ascending=True)
df

,Word,Count
0,improvements,1562
7,bug,1673
14,related,2762
10,issue,5431
20,request,10150
19,feature,10656


In [0]:
fig = go.Figure()

fig.add_trace(go.Bar(
    y=df['Word'],
    x=df['Count'],
    orientation='h',
    marker=dict(color='yellowgreen'),
    marker_line_color='rgb(8, 48, 107)',
    marker_line_width=1.5,
    opacity=0.6,
    width=0.5,
))

fig.update_layout(
    title='Frequency Distribution of Words in Description',
    yaxis=dict(tickangle=0),
)

fig.show()

In [0]:
pandas_df = new_df.select('name').toPandas()

In [0]:
# converting to lowercase
pandas_df['name'] = pandas_df['name'].astype(str).str.lower()

# converting to tokens
regexp = RegexpTokenizer('\w+')
pandas_df['name_token']=pandas_df['name'].apply(regexp.tokenize)
nltk.download('stopwords')

# stopwords
stopwords1 = nltk.corpus.stopwords.words("english")
my_stopwords = ['add','good','first','priority','task','help','channels','wanted','documentation']
stopwords1.extend(my_stopwords)
pandas_df['name_token'] = pandas_df['name_token'].apply(lambda x: [item for item in x if item not in stopwords1])

# infrequent words
pandas_df['name_str'] = pandas_df['name_token'].apply(lambda x: ' '.join([item for item in x if len(item)>2]))

# creating list of words
all_words_name = ' '.join([word for word in pandas_df['name_str']])

# tokenizing all words
nltk.download('punkt')
tokenized_words_name = nltk.tokenize.word_tokenize(all_words_name)

# frequency distribution
fdist_name = FreqDist(tokenized_words_name)
pandas_df['name_fdist'] = pandas_df['name_token'].apply(lambda x: ' '.join([item for item in x if fdist_name[item] >= 1]))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
all_words_desc = ' '.join([word for word in pandas_df['name_fdist']])

In [0]:
# fd table
words = nltk.word_tokenize(all_words_desc)
fd = FreqDist(words)
data_fd = list(fd.items())
df = pd.DataFrame(data_fd, columns=['Word', 'Count'])
df = df.sort_values(by='Count', ascending=False).head(6)
df = df.sort_values(by='Count', ascending=True)
df

,Word,Count
64,question,826
85,feature,1178
5,issue,1230
59,type,1239
6,enhancement,8843
1,bug,10454


In [0]:
fig = go.Figure()

fig.add_trace(go.Bar(
    y=df['Word'],
    x=df['Count'],
    orientation='h',
    marker=dict(color='red'),
    marker_line_color='rgb(8, 48, 107)',
    marker_line_width=1.5,
    opacity=0.6,
    width=0.5,
))

fig.update_layout(
    title='Frequency Distribution of Words in Labels',
    yaxis=dict(tickangle=0),
)

fig.show()

In [0]:
bug_rows = new_df.filter(col('name').like('%bug%'))
k = bug_rows.toPandas()
k['name'].unique()

Out[177]: array(['bug', 'typebug', 'issue bug', 'cbug', 'type bug', 'kindbug',
       'persistentbug', 'bug v', 'docbug', 'bugreport', 'bug  fix',
       'issuebug', 'type  report  bug', 'bug report', 'bugfix',
       'type bug report', 'goodfirstbug', 'kindbugreport', 'bugconfirmed',
       'bugsolution proposed', 'tbug', 'a bug', 'major bug',
       'template bug', 'bug bug', 'kind bug', 'bug tool', 'bug bash',
       'bbug', 'impact bug', 't bug', 'unconfirmed bug', 'feature bug',
       'recipe bug', 'potential bug', 'codebug', 'bugunconfirmed',
       'confirmed bug', 'typedebug', 'typebugreport', 'bugbehaviour',
       'bug invalid', 'bug unconfirmed', 'bug accepted', 'type  bug',
       'critical bug', 'bug sweatsmile', 'type product bug', 'p bug',
       'bugregression', 'isbug', 'bug  issue', 'beetle bug',
       'upstream bug', 'adebuginfo', 'fdebuggervisualizer', 'bugunknown',
       'bugs', 'bugverified', 'bug boom', 'ladybeetle  bug', 'sitebug',
       'bug  brokenfeature'

In [0]:
new_df.filter(col('name').like('%priority%'))

Out[206]: DataFrame[name: string, description: string]

In [0]:
new_df = new_df.withColumn('name', when(col('name').like('%bug%'),'bug').otherwise(col('name')))

In [0]:
new_df = new_df.withColumn('name', when(col('name').like('%enhancement%'),'feature').otherwise(col('name')))

In [0]:
new_df = new_df.withColumn('name', when(col('name').like('%priority%'),'feature').otherwise(col('name')))

+-----------------+
|(name = priority)|
+-----------------+
|            false|
|            false|
|            false|
|            false|
|            false|
|            false|
|            false|
|            false|
|            false|
|            false|
|            false|
|            false|
|            false|
|            false|
|            false|
|            false|
|            false|
|            false|
|            false|
|            false|
+-----------------+
only showing top 20 rows



In [0]:
new_df = new_df.withColumn('name', when(col('name').like('%request%'),'request').otherwise(col('name')))

In [0]:
name_group_1 = new_df.groupBy(new_df.name).count().orderBy("count", ascending=False).limit(10).toPandas()

In [0]:
fig = go.Figure([go.Bar(x=name_group['name'], y=name_group['count'],marker=dict(color='darkmagenta'))])
fig.update_traces( marker_line_color='rgb(8,48,107)',
                  marker_line_width=1.5, opacity=0.6,width=0.5)
fig.update_layout(title_text='Label Counts',xaxis=dict(tickangle=90))
fig.show()

In [0]:
fig = go.Figure([go.Bar(x=name_group_1['name'], y=name_group_1['count'],marker=dict(color='blue'))])
fig.update_traces( marker_line_color='rgb(8,48,107)',
                  marker_line_width=1.5, opacity=0.6,width=0.5)
fig.update_layout(title_text='Label Counts',xaxis=dict(tickangle=90))
fig.show()

In [0]:
new_df.filter((new_df.name != 'bug') & (new_df.name != 'feature') & (new_df.name != 'request')).count()/new_df.count()

Out[211]: 0.5689183378288574